# Document principal du projet

## Création de la base de données


On se sert directement du fichier *src/services/init/initialisation.json*. Ce fichier contient une liste de codes-barres que l'on utilise pour instancier la base de données *src/database/barcodes.json*.

In [1]:
import os
print(os.getcwd())
# On est au mauvais endroit donc on set up le working directory correctement 
os.chdir(os.path.dirname(os.getcwd()))
print(os.getcwd())



/home/onyxia/work/Projet-python-pour-la-data-science---ENSAE-Paris/src
/home/onyxia/work/Projet-python-pour-la-data-science---ENSAE-Paris


In [2]:
from src.services.barcodes import Barcodes
import os

# On instancie la base de données si cela n'a pas déjà été fait
bar = Barcodes()
bar.initialiser_database()

In [3]:
# Chemin actuel
current_dir = os.getcwd()
print(f"Répertoire actuel : {current_dir}")

# Revenir au répertoire parent

# Nouveau chemin
new_dir = os.getcwd()
print(f"Répertoire après retour en arrière : {new_dir}")

Répertoire actuel : /home/onyxia/work/Projet-python-pour-la-data-science---ENSAE-Paris
Répertoire après retour en arrière : /home/onyxia/work/Projet-python-pour-la-data-science---ENSAE-Paris


In [ ]:
from src.services.traitement import Traitement
# Création d'une instance
traitement = Traitement()

# Application de la transformation
#traitement.appliquer_transformation_categories() Probleme il faut avoir les produits en lignes et pas en colonnes

# Affichage des données transformées
print(traitement.df.head())
#df = pd.DataFrame.from_dict(data, orient='index')

                                           3073781055016  \
Nom                                    Fagotin tradition   
Categorie                                     en:dairies   
Nutriscore                                             d   
Novascore   en:4-ultra-processed-food-and-drink-products   
Ecoscore                                              47   

                                           3564707128117  \
Nom                              Ricotta bio 8% Mat. Gr.   
Categorie                                     en:dairies   
Nutriscore                                             c   
Novascore   en:4-ultra-processed-food-and-drink-products   
Ecoscore                                              60   

                                 3564700583135         3596710495979  \
Nom         Fromage Le Carre Au Lait Pasterise     Gouda tranchettes   
Categorie                           en:dairies            en:dairies   
Nutriscore                                   d                